In [1]:
import math
import csv
from numpy import genfromtxt
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict
import pandas as pd

%matplotlib inline
np.random.seed(1)

In [2]:
# the original dataset as numpy matrix 
XY_orig=genfromtxt('churn_pred_12.csv', delimiter=',') 
XY_orig=np.matrix(XY_orig)   

# taking out the X portion
X_orig=XY_orig[1:50001, 1:17].T   
X_orig_max = X_orig.max(1)          
X_orig=X_orig/X_orig_max
print ("X_orig shape: " + str(X_orig.shape))

# taking out the Y portion
Y_orig=XY_orig[1:50001, 17].T       

# training set
X_train=X_orig[:, 0:45000]          
Y_train=Y_orig[:, 0:45000]          

# test set
X_test=X_orig[:, 45000:50001]       
Y_test=Y_orig[:, 45000:50001]       

# convert all examples to numpy arrays
X_train=np.array(X_train)           
X_test=np.array(X_test)
Y_train=np.array(Y_train)
Y_test=np.array(Y_test)

# convert to one-hot representations
Y_train=Y_train.astype(int)         
Y_test=Y_test.astype(int)
Y_train=convert_to_one_hot(Y_train, 2)
Y_test=convert_to_one_hot(Y_test, 2)

# printing out dimensions
print ("number of training examples = " + str(X_train.shape[1]))      
print ("number of test examples = " + str(X_test.shape[1]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

# printing out details
print("NaNs exist in training set: " + str(np.isnan(X_train).any()))
print("NaNs exist in test set: " + str(np.isnan(X_test).any()))

X_orig shape: (16, 50000)
number of training examples = 45000
number of test examples = 5000
X_train shape: (16, 45000)
Y_train shape: (2, 45000)
X_test shape: (16, 5000)
Y_test shape: (2, 5000)
NaNs exist in training set: False
NaNs exist in test set: False


In [3]:
# taking out the mobile numbers 
df_mobile_nos=pd.read_csv("churn_pred_12.csv") 
mobile_nos=list(df_mobile_nos["mobile_no"])

# users with differently predicted retentions
pred_0=[]
pred_1=[]

In [4]:
def create_placeholders(n_x, n_y):

    X = tf.placeholder(tf.float32, [n_x, None])
    Y = tf.placeholder(tf.float32, [n_y, None])
    
    return X, Y

In [5]:
def initialize_parameters():
    
    # keeping results consistent
    tf.set_random_seed(1)                   
        
    # defining the neural network's architecture
    W1 = tf.get_variable("W1", [20, 16], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b1 = tf.get_variable("b1", [20, 1], initializer = tf.zeros_initializer())
    
    W2 = tf.get_variable("W2", [25, 20], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b2 = tf.get_variable("b2", [25, 1], initializer = tf.zeros_initializer())
    
    W3 = tf.get_variable("W3", [25, 25], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b3 = tf.get_variable("b3", [25, 1], initializer = tf.zeros_initializer())
    
    W4 = tf.get_variable("W4", [25, 25], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b4 = tf.get_variable("b4", [25, 1], initializer = tf.zeros_initializer())
    
    W5 = tf.get_variable("W5", [20, 25], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b5 = tf.get_variable("b5", [20, 1], initializer = tf.zeros_initializer())
    
    W6 = tf.get_variable("W6", [2, 20], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b6 = tf.get_variable("b6", [2, 1], initializer = tf.zeros_initializer())

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3,
                  "W4": W4,
                  "b4": b4,
                  "W5": W5,
                  "b5": b5,
                  "W6": W6,
                  "b6": b6}
    
    return parameters

In [6]:
def forward_propagation(X, parameters):
    
    # retrieving the parameters from the dictionary 'parameters' 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    W4 = parameters['W4']
    b4 = parameters['b4']
    W5 = parameters['W5']
    b5 = parameters['b5']
    W6 = parameters['W6']
    b6 = parameters['b6']
                                         
    Z1 = tf.add(tf.matmul(W1, X), b1)                                              
    A1 = tf.nn.relu(Z1)   
    
    Z2 = tf.add(tf.matmul(W2, A1), b2)                                             
    A2 = tf.nn.relu(Z2)  
    
    Z3 = tf.add(tf.matmul(W3, A2), b3) 
    A3 = tf.nn.relu(Z3)
    
    Z4 = tf.add(tf.matmul(W4, A3), b4)
    A4 = tf.nn.relu(Z4)
    
    Z5 = tf.add(tf.matmul(W5, A4), b5)
    A5 = tf.nn.relu(Z5)
    
    Z6 = tf.add(tf.matmul(W6, A5), b6)
    
    return Z6

In [7]:
def compute_cost(Z6, Y):
    
    # to fit the tensorflow requirement for tf.nn.softmax_cross_entropy_with_logits(...,...)
    logits = tf.transpose(Z6)
    labels = tf.transpose(Y)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    
    return cost

In [8]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.01,
          num_epochs = 500, minibatch_size = 64, print_cost = True):
    
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep consistent results
    seed = 3                                          # to keep consistent results
    (n_x, m) = X_train.shape                          # n_x: input size, m : number of examples in the train set
    n_y = Y_train.shape[0]                            # n_y: output size
    costs = []                                        # to keep track of the cost
    
    # creating Placeholders of shape (n_x, n_y)
    X, Y = create_placeholders(n_x, n_y)

    # initializing parameters
    parameters = initialize_parameters()
    
    # forward propagation: building the forward propagation in the tensorflow graph
    Z6 = forward_propagation(X, parameters)
    
    # cost function: adding cost function to tensorflow graph
    cost = compute_cost(Z6, Y)
    
    # backpropagation: defining the tensorflow AdamOptimizer
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    
    # initializing all the variables
    init = tf.global_variables_initializer()

    # starting the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # running the initialization
        sess.run(init)
        
        # training loop
        for epoch in range(num_epochs):

            epoch_cost = 0.                       
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

            for minibatch in minibatches:

                # selecting a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                
                _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
                
                epoch_cost += minibatch_cost / num_minibatches

            # printing the cost every epoch
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)

        # saving the parameters in a variable
        parameters = sess.run(parameters)
        
        print ("Parameters have been trained!")
        print()

        # accuracy
        correct_prediction = tf.equal(tf.argmax(Z6), tf.argmax(Y))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        
        print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
        print ("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
        print()
        
        # predictions against test-set
        predicted0=tf.equal(tf.argmax(Z6), 0)
        predicted1=tf.equal(tf.argmax(Z6), 1)
        
        predicted_zeros=list(predicted0.eval({X: X_test}))
        len_predicted_zeros=len(predicted_zeros)
        for i in range(0, len_predicted_zeros):
            if(predicted_zeros[i]==True): pred_0.append(mobile_nos[i])
                
        predicted_ones=list(predicted1.eval({X: X_test}))
        len_predicted_ones=len(predicted_ones)
        for i in range(0, len_predicted_ones):
            if(predicted_ones[i]==True): pred_1.append(mobile_nos[i])
        
        return parameters

In [9]:
parameters = model(X_train, Y_train, X_test, Y_test)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Cost after epoch 0: 0.441296
Cost after epoch 100: 0.422494
Cost after epoch 200: 0.423564
Cost after epoch 300: 0.421459
Cost after epoch 400: 0.421879
Parameters have been trained!

Train Accuracy: 0.8110222
Test Accuracy: 0.8038



In [10]:
print(str(len(pred_0)) + " users predicted not to return in the next 14 days are:\n" + str(pred_0))
print()

print(str(len(pred_1)) + " users predicted to return in the next 14 days are:\n" + str(pred_1))
print()

2962 users predicted not to return in the next 14 days are:
[1783992336, 1966661274, 1736430814, 1718917022, 1920298687, 1688801001, 1714863112, 1942104116, 1713779696, 1762093080, 1914637887, 1788099101, 1736323450, 1918062493, 1711415320, 1718502394, 1727450791, 1307568656, 1716007227, 1785819662, 1761754760, 1610745154, 1611863094, 1820141441, 1709793038, 1873505251, 1829116353, 1975974454, 1717606333, 1303169644, 1991054784, 1911320628, 1912798393, 1719143774, 1783868680, 1982499884, 1630399780, 1709102585, 1749739254, 1314436613, 1713922611, 1703412850, 1749035177, 1714881128, 1922554170, 1791301466, 1838685547, 1920650569, 1842252965, 1911621926, 1820332339, 1773073202, 1405188535, 1711968022, 1324319048, 1705314710, 1849260287, 1741413141, 1859506257, 1626963524, 1725262509, 1914776969, 1303226075, 1753851250, 1748781729, 1720342317, 1630952486, 1733152245, 1719122935, 1933948610, 1639340074, 1874809642, 1964297039, 1814184897, 1701398899, 1787623098, 1748736352, 1722954514, 131